Analyse big corpus of texts which already have pre-estimated embeddings.

Key problems - long time for UMAP & HDBSDAN.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install bertopic
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install --upgrade cupy-cuda11x -f https://pip.cupy.dev/aarch64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 66.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

In [ ]:
# !pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from tqdm import tqdm

# from nltk.corpus import stopwords
# from string import punctuation

# import nltk
# nltk.download('stopwords')

In [ ]:
# Topic model
from bertopic import BERTopic
# Sentence model
# from sentence_transformers import SentenceTransformer
# Load models from ciML
# from umap import UMAP
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN

from sklearn.utils.random import sample_without_replacement

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/TG news channels/temp/"

In [ ]:
# load data
df = pd.read_csv(path+'df_filtered.csv')
df_cross = pd.read_csv(path+'df_cross.csv')

# initialize docs and embeddings
docs = df['cleaned_message']
embeddings = df.iloc[:, -384:].to_numpy()

<ipython-input-11-c5bb20b7e23f>:2: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+'df_filtered.csv')


In [ ]:
df.shape, docs.shape, embeddings.shape

((127986, 399), (127986,), (127986, 384))

In [ ]:
# Initiate BERTopic
umap_model = UMAP(n_components=15, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

topic_model = BERTopic(umap_model=umap_model, language="russian", n_gram_range=(1,3),  \
                       calculate_probabilities=True, verbose = True)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(docs, embeddings = embeddings)

2023-06-23 16:10:50,322 - BERTopic - Reduced dimensionality
2023-06-23 19:43:23,170 - BERTopic - Clustered reduced embeddings
